### Tests for Applying Gains Matrix

In [1]:
import cupy as cp
import numpy as np
from utils import *
from zp_puregpu_funcs_py import *
from invcov import *

hera_cal is not installed. Some gridding features unavailable.
pyfof is not installed. Some gridding features unavailable.


In [16]:
apply_gains_to_mat?
# undo_zeropad?

In [3]:
summarize_benchmark_results?

In [4]:
#simulation params relevant for testing application of gains to a matrix
n_ant = 4
n_bl = 30
n_gains = 2*n_bl
n_eig = 3
xp = cp  #run things on the gpu using cupy

#this might be the easiest (and most general) way to devise an edges
# array, though we hard code an ex. edges array to be sure it fits
# the desired format of having no odd entries.
edges = (xp.unique(xp.random.randint(1, n_bl/2-1, size = n_ant)*2))
edges = xp.concatenate((xp.array([0]), edges, xp.array([n_bl])))
print(f"The edges of the redundant blocks have indices{edges}")

#some random noise, diffuse, and source covariance matrices
xp = cp
sim_diff_mat = xp.random.rand(n_bl, n_eig, dtype = 'float64')
sim_gains = cp.random.rand(n_gains, dtype = 'float64') #Re/Im split + ant1 & ant 2 = 4*n_ant
ant_1_array = cp.arange(n_bl//2)
ant_2_array = cp.arange(n_bl//2)

#zeropad the noise, diff, source mats
zp_sim_diff_mat, largest_block, n_blocks = zeroPad(sim_diff_mat, edges, return_inv=False)

#Need to reshape to give an extra dimension of n_blocks to be compatible with inv cov routine
sim_diff_mat_3d = zp_sim_diff_mat.reshape(n_blocks, largest_block, n_eig)

The edges of the redundant blocks have indices[ 0  2  4 12 14 30]


In [5]:
applied_gains = apply_gains_to_mat(sim_gains, sim_diff_mat_3d, edges, ant_1_array, ant_1_array, cp, True)

In [6]:
print(f"Input mat has shape {sim_diff_mat_3d.shape}")
print()
print(f"Gains applied mat has shape {applied_gains.shape}")
print()
print(f"The mat with gains applied looks like:\n\n{applied_gains}")

Input mat has shape (5, 16, 3)

Gains applied mat has shape (5, 16, 3)

The mat with gains applied looks like:

[[[6.14775437e-01 6.04644313e-01 1.01457219e+00]
  [8.77434152e-01 4.37661657e-01 1.46935036e-01]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00]]

 [[6.57629330e-02 3.83288803e-03 2.31902753e-02]
  [5.26389294e-02 3.88585817e-02 5.23537072e-02]
  [0

#### Random Debugging tests

In [7]:
m = np.random.rand(2, 5, 3)
g = np.random.rand(10)

gs = g[:, None] * g[:, None]
gs = gs.reshape(m.shape[0], m.shape[1], 1)

print(gs.shape)
print(m.shape)
print(g.shape)
print(m)
print(m[:, ::2])
out = gs[:, ::2] * m[:, ::2]

(2, 5, 1)
(2, 5, 3)
(10,)
[[[0.53068928 0.99600573 0.08743734]
  [0.65132888 0.94854942 0.2491223 ]
  [0.35891228 0.26268738 0.88193327]
  [0.76316287 0.15499556 0.10642114]
  [0.75839618 0.01682198 0.78419829]]

 [[0.34649647 0.48638336 0.44356207]
  [0.44438274 0.41192295 0.68454618]
  [0.93237129 0.60344295 0.25770383]
  [0.38509288 0.80383794 0.19093417]
  [0.5434582  0.27532584 0.78401389]]]
[[[0.53068928 0.99600573 0.08743734]
  [0.35891228 0.26268738 0.88193327]
  [0.75839618 0.01682198 0.78419829]]

 [[0.34649647 0.48638336 0.44356207]
  [0.93237129 0.60344295 0.25770383]
  [0.5434582  0.27532584 0.78401389]]]


In [8]:
a = np.arange(5)
b = np.arange(5,10)
c = np.random.rand((a.shape[0] + b.shape[0]))
print(c)
d = c[a]
e = c[b]

print(d)
print()
print(e)
print()
print(np.concatenate((d, e)))
print()
aa = np.concatenate((d, e))

print(aa[:int(len(aa)/2)])

print(aa[int(len(aa)/2):])

[0.52142401 0.51809513 0.28015706 0.97188828 0.07147487 0.06944068
 0.98387057 0.52674566 0.2883073  0.18897522]
[0.52142401 0.51809513 0.28015706 0.97188828 0.07147487]

[0.06944068 0.98387057 0.52674566 0.2883073  0.18897522]

[0.52142401 0.51809513 0.28015706 0.97188828 0.07147487 0.06944068
 0.98387057 0.52674566 0.2883073  0.18897522]

[0.52142401 0.51809513 0.28015706 0.97188828 0.07147487]
[0.06944068 0.98387057 0.52674566 0.2883073  0.18897522]


#### Check ZP computes the same thing as corrcal

In [9]:
#Current corrcal gain application function
def apply_gains_to_mat_corrcal(gains, mat, ant_1_array, ant_2_array):
    """Apply a gain-like matrix to a provided matrix."""
    complex_gains = gains[::2] + 1j*gains[1::2]
    gain_mat = (
        complex_gains[ant_1_array,None] * complex_gains[ant_2_array,None].conj()
    )
    out = np.zeros_like(mat)
    out[::2] = gain_mat.real * mat[::2] - gain_mat.imag * mat[1::2]
    out[1::2] = gain_mat.imag * mat[::2] + gain_mat.real * mat[1::2]
    return out

In [10]:
#simulation params relevant for testing application of gains to a matrix
n_ant = 15
n_bl = 2*n_ant
n_gains = 4*n_ant
n_eig = 3
xp = cp  #run things on the gpu using cupy

#this might be the easiest (and most general) way to devise an edges
# array, though we hard code an ex. edges array to be sure it fits
# the desired format of having no odd entries.
edges = (xp.unique(xp.random.randint(1, n_bl/2-1, size = 3)*2))
edges = xp.concatenate((xp.array([0]), edges, xp.array([n_bl])))
print(f"The edges of the redundant blocks have indices{edges}")

#some random noise, diffuse, and source covariance matrices
xp = cp
sim_diff_mat = xp.random.rand(n_bl, n_eig, dtype = 'float64')
sim_gains = cp.random.rand(n_gains, dtype = 'float64') #Re/Im split + ant1 & ant 2 = 4*n_ant
ant_1_array = cp.arange(n_ant)
ant_2_array = cp.arange(n_ant, 2*n_ant)

#zeropad the noise, diff, source mats
zp_sim_diff_mat, largest_block, n_blocks = zeroPad(sim_diff_mat, edges, return_inv=False)

#Need to reshape to give an extra dimension of n_blocks to be compatible with inv cov routine
sim_diff_mat_3d = zp_sim_diff_mat.reshape(n_blocks, largest_block, n_eig)


The edges of the redundant blocks have indices[ 0  4 12 16 30]


In [11]:
# out_corrcal = apply_gains_to_mat_corrcal(sim_gains, sim_diff_mat, ant_1_array, ant_2_array)
out_gpu = apply_gains_to_mat(sim_gains, sim_diff_mat_3d, edges, ant_1_array, ant_2_array, cp)
out_gpu_corrcal = apply_gains_to_mat(sim_gains, sim_diff_mat, edges, ant_1_array, ant_2_array, cp, False)
out_gpu_resh = undo_zeropad(out_gpu, edges, cp)

In [12]:
#should return True if my routine computes the same thing as the current corrcal routine
print(cp.allclose(out_gpu_resh, out_gpu_corrcal))

True


#### Timing test for np vs cp versions

In [13]:
#simulation params relevant for testing application of gains to a matrix
n_bl = 240000  # something like 2*N_ant^2 for Re/Im split
n_ant = 512
n_gains = 2*n_bl  #one for each antenna and one for each Re and Im part
n_eig = 3
xp = cp  #run things on the gpu using cupy

#this might be the easiest (and most general) way to devise an edges
# array, though we hard code an ex. edges array to be sure it fits
# the desired format of having no odd entries.
if xp == cp:

    edges = (xp.unique(xp.random.randint(1, n_bl/2-1, size = n_ant)*2))
    edges = xp.concatenate((xp.array([0]), edges, xp.array([n_bl])))
    print(f"The edges of the redundant blocks have indices{edges}")

    #some random noise, diffuse, and source covariance matrices
    sim_diff_mat = xp.random.rand(n_bl, n_eig, dtype = 'float64')
    sim_gains = cp.random.rand(n_gains, dtype = 'float64') #Re/Im split + ant1 & ant 2 = 4*n_ant
    
    #leave as n_bl/2 since indexex into cplex gains which are not Re/Im split
    ant_1_array = cp.arange(n_bl//2) 
    ant_2_array = cp.arange(n_bl//2)

    #zeropad the noise, diff, source mats
    zp_sim_diff_mat, largest_block, n_blocks = zeroPad(sim_diff_mat, edges, return_inv=False)

    #Need to reshape to give an extra dimension of n_blocks to be compatible with inv cov routine
    sim_diff_mat_3d = zp_sim_diff_mat.reshape(n_blocks, largest_block, n_eig)

elif xp == np:
    edges = (xp.unique(xp.random.randint(1, n_bl/2-1, size = n_ant)*2))
    edges = xp.concatenate((xp.array([0]), edges, xp.array([n_bl])))
    print(f"The edges of the redundant blocks have indices{edges}")

    #some random noise, diffuse, and source covariance matrices
    sim_diff_mat = xp.random.rand(n_bl, n_eig).astype('float64')
    sim_gains = xp.random.rand(n_gains).astype('float64') #Re/Im split + ant1 & ant 2 = 4*n_ant
    ant_1_array = xp.arange(n_bl//2)
    ant_2_array = xp.arange(n_bl//2)

    #zeropad the noise, diff, source mats
    # zp_sim_diff_mat, largest_block, n_blocks = zeroPad(sim_diff_mat, edges, return_inv=False)

    #Need to reshape to give an extra dimension of n_blocks to be compatible with inv cov routine
    # sim_diff_mat_3d = zp_sim_diff_mat.reshape(n_blocks, largest_block, n_eig)

print(len(edges))

The edges of the redundant blocks have indices[     0   1322   1580   2908   3142   3770   4134   4884   5172   5322
   6326   6746   7422   7426   8096   8104   9260   9364   9618   9950
  10416  10778  10842  10962  11308  12200  12508  12586  12912  12956
  13950  14254  15490  17514  18108  18252  18950  18974  19248  19948
  20028  20124  20410  21932  22400  22864  23022  23228  23410  23644
  23668  23922  24764  24832  25564  26090  26152  26600  26908  27416
  27898  28360  28846  29246  30094  30726  31040  31340  31352  31566
  31660  32228  32248  32788  33078  33198  33976  34062  34192  34606
  34870  35004  36172  36296  36580  37108  38032  38136  38278  38592
  38688  38822  39402  40512  40954  42016  42472  42898  43398  43866
  44188  44202  44678  44696  44778  45938  46200  46332  46910  47048
  47814  48220  48724  49350  49756  50412  50444  50540  50724  51698
  51918  52426  54340  55424  55452  57526  57674  58546  58708  58842
  59164  59370  60596  60726  6

In [14]:
print(type(sim_diff_mat))
print(type(sim_gains))
summarize_benchmark_results(apply_gains_to_mat, sim_gains, sim_diff_mat, edges, ant_1_array, ant_2_array, xp, False)

<class 'cupy.ndarray'>
<class 'cupy.ndarray'>
Time on cpu: 0.000792s
Time on gpu: 0.000826s


In [15]:
print(type(sim_diff_mat))
print(type(sim_gains))
summarize_benchmark_results(apply_gains_to_mat, sim_gains, sim_diff_mat_3d, edges, ant_1_array, ant_2_array, xp, True)

<class 'cupy.ndarray'>
<class 'cupy.ndarray'>
Time on cpu: 0.002296s
Time on gpu: 0.003999s
